# Part 2 - Branch Versioning with Parcel Fabric

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Part-2---Branch-Versioning-with-Parcel-Fabric" data-toc-modified-id="Part-2---Branch-Versioning-with-Parcel-Fabric-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Part 2 - Branch Versioning with Parcel Fabric</a></span><ul class="toc-item"><li><ul class="toc-item"><li><span><a href="#Components-of-a-Parcel-Fabric-Feature-Service" data-toc-modified-id="Components-of-a-Parcel-Fabric-Feature-Service-1.0.1"><span class="toc-item-num">1.0.1&nbsp;&nbsp;</span>Components of a Parcel Fabric Feature Service</a></span></li><li><span><a href="#Parcel-editing-workflow" data-toc-modified-id="Parcel-editing-workflow-1.0.2"><span class="toc-item-num">1.0.2&nbsp;&nbsp;</span>Parcel editing workflow</a></span></li><li><span><a href="#Access-branch-versions-via-Version-Management-Server" data-toc-modified-id="Access-branch-versions-via-Version-Management-Server-1.0.3"><span class="toc-item-num">1.0.3&nbsp;&nbsp;</span>Access branch versions via Version Management Server</a></span><ul class="toc-item"><li><span><a href="#Get-all-versions" data-toc-modified-id="Get-all-versions-1.0.3.1"><span class="toc-item-num">1.0.3.1&nbsp;&nbsp;</span>Get all versions</a></span></li><li><span><a href="#Create-a-version" data-toc-modified-id="Create-a-version-1.0.3.2"><span class="toc-item-num">1.0.3.2&nbsp;&nbsp;</span>Create a version</a></span></li></ul></li><li><span><a href="#Access-the-Version-Management-Server-through-the-Parcel-Fabric-FeatureLayerCollection-object" data-toc-modified-id="Access-the-Version-Management-Server-through-the-Parcel-Fabric-FeatureLayerCollection-object-1.0.4"><span class="toc-item-num">1.0.4&nbsp;&nbsp;</span>Access the Version Management Server through the Parcel Fabric <code>FeatureLayerCollection</code> object</a></span></li><li><span><a href="#Branch-Versioning-Edit-Sessions" data-toc-modified-id="Branch-Versioning-Edit-Sessions-1.0.5"><span class="toc-item-num">1.0.5&nbsp;&nbsp;</span>Branch Versioning Edit Sessions</a></span><ul class="toc-item"><li><span><a href="#Recocile,-Post-and-Delete-the-version" data-toc-modified-id="Recocile,-Post-and-Delete-the-version-1.0.5.1"><span class="toc-item-num">1.0.5.1&nbsp;&nbsp;</span>Recocile, Post and Delete the version</a></span></li></ul></li><li><span><a href="#API-Ref-Documentation" data-toc-modified-id="API-Ref-Documentation-1.0.6"><span class="toc-item-num">1.0.6&nbsp;&nbsp;</span>API Ref Documentation</a></span></li></ul></li></ul></li></ul></div>

### Components of a Parcel Fabric Feature Service
A Parcel Fabric feature service consists of at least 5 endpoints (SOEs)
- Mapping (.../MapServer)
- Feature Access (.../FeatureServer)
- Version Management (.../VersionManagementServer)
- Validation (.../ValidationServer)
- Parcel Fabric (../ParcelFabricServer)

Most Parcel Fabric editing operations will make use of the Feature Access, Version Management and Parcel Fabric endpo
#ints.  

### Parcel editing workflow
At a high level, a typical parcel editing workflow will look like:

1. Create a branch version to isolate edits from the default version. (Version Management)
2. Start an edit session (Version Management)
3. Create a new parcel Record feature (Feature Access)
4. Edit one or more parcels (merge, divide, copy lines, etc.) (Parcel Fabric)
5. Reconcile the current branch version with the default version. (Version Management)
6. Post changes from the current version to the default version. (Version Management)
7. Stop the edit session. (Version Management)
8. Delete the version. (Version Management)

Based on this example, it is obvious that understanding versioned, multi-user editing is critical.  The following notebooks will demonstrate how to list versions, create versions, start and stop edit sessions and reconcile and post edits.

In [1]:
from arcgis import GIS
from arcgis.features import _version

In [2]:
base_server_url = "https://myserver.domain.com/web_adaptor/rest/services/WashingtonCountyLSA/"
gis = GIS("https://myserver.domain.com/web_adaptor/", "username", "pass.word", verify_cert=False)

### Access branch versions via Version Management Server

In [14]:
from arcgis.features._version import VersionManager

version_management_server_url = f"{base_server_url}/VersionManagementServer"
vms = VersionManager(version_management_server_url, gis)
vms.properties

{
  "name": "Version Management Server",
  "type": "Map Server Extension",
  "defaultVersionName": "sde.DEFAULT",
  "defaultVersionGuid": "{BD3F4817-9A00-41AC-B0CC-58F78DBAE0A1}",
  "capabilities": {
    "supportsConflictDetectionByAttribute": true,
    "supportsPartialPost": true,
    "supportsDifferencesFromMoment": true,
    "supportsDifferencesWithLayers": true
  }
}

#### Get all versions

In [15]:
versions = vms.all
versions

[<Version sde.DEFAULT @ {BD3F4817-9A00-41AC-B0CC-58F78DBAE0A1}>,
 <Version admin.waco_lsa_analyze @ {B2752859-D811-4C9C-B6F9-9650713CFDB2}>,
 <Version admin.gen_fabric_links @ {80FF6200-ACB2-477F-B79C-188A61EB1C9A}>,
 <Version admin.generate_fabric_links @ {D31E7834-134E-4C91-A54B-736D78714172}>]

#### Create a version

In [16]:
new_version_name = "fabric_editor_1"
vms.create(new_version_name)

True

### Access the Version Management Server through the Parcel Fabric `FeatureLayerCollection` object

The `versions` property in a parcel fabric `FeatureLayerCollection` (FLC) creates a `VersionManager` object to create, update and use versions.  

The `FeatureServer` endpoint is used to create a `FeatureLayerCollection`.

In [17]:
from arcgis.features.layer import FeatureLayerCollection
parcel_fabric_feature_server_url = f"{base_server_url}/FeatureServer"
parcel_fabric_flc = FeatureLayerCollection(parcel_fabric_feature_server_url, gis)

# print the version names from the FLC's versions property:
vms_from_flc = parcel_fabric_flc.versions
[print(v.properties.versionName) for v in vms_from_flc.all]

sde.DEFAULT
admin.waco_lsa_analyze
admin.gen_fabric_links
admin.generate_fabric_links
admin.fabric_editor_1


[None, None, None, None, None]

### Branch Versioning Edit Sessions

A branch versioning edit session is act of obtaining shared and exclusive locks on the feature class to prevent corruption in the branch version. Calling `version.startReading` will set a shared lock on the version which prevents another session from obtaining an exclusive lock.  Other sessions can still access the version as read-only. Calling `version.startEditing` will set the exclusive lock which will prevent write access and write access to the version.

Keeping track of where one is within the edit sessions is made simple with a built in context manager.

In [18]:
from arcgis.features import _parcel
parcel_fabric_manager_url = f"{base_server_url}/ParcelFabricServer"

# start a 'read' session to acquire a shared lock and 
# get a branch version by its name
with vms.get("admin.generate_fabric_links", "read") as version:
    parcel_fabric_manager = _parcel.ParcelFabricManager(
        parcel_fabric_manager_url, 
        gis, 
        version, 
        parcel_fabric_flc)

    # do parcel fabric or other feature service editing within the version
    # i.e. parcel_fabric_manager.copy_lines_to_parcel_type(...)

parcel_fabric_manager.properties

{
  "name": "Parcel Fabric Server",
  "type": "Map Server Extension",
  "ParcelFabricLayers": 1
}

#### Recocile, Post and Delete the version

When editing is complete, the new features can be posted from the new branch version to the default version.  In this workflow, Reconcile must occur first.  Once posted, the version can optionally be deleted.

In [19]:
version = vms.get(f"admin.{new_version_name}")

# version.reconcile()
# version.post

version.delete()

True

### API Ref Documentation

- [ArcGIS Python API - Version Manager](https://developers.arcgis.com/python/api-reference/arcgis.features.managers.html#versionmanager)
- [ArcGIS Python API - Parcel Fabric Manager](https://developers.arcgis.com/python/api-reference/arcgis.features.managers.html#parcelfabricmanager)
- [ArcGIS REST API - VersionManagementServer](https://developers.arcgis.com/rest/services-reference/enterprise/version-management-service.htm)
- [ArcGIS REST API - ParcelFabricServer](https://developers.arcgis.com/rest/services-reference/enterprise/overview-of-parcel-fabric-sevices.htm)
- [ArcGIS Pro - Branch Versioning Scenarios](https://pro.arcgis.com/en/pro-app/latest/help/data/geodatabases/overview/branch-version-scenarios.htm)